# 04 - Neural Networks

#### Let's get to the real deal ML - neural nets, aka Multilayer Perceptrons.

We'll **build a minimal neural net from scratch** and use it to predict a single number from a synthetic dataset.  

**Architecture**: 2 input vars, 1 layer of 4 hidden neurons, one continuous output

## What we'll do
0. Setup & make a tiny synthetic dataset (2 features, 1 target)  
1. Define the loss function (**MSE**)  
2. Define the **forward pass** (with bias via input augmentation)  
3**Backprop**: derive gradients and implement training loop (next, after you confirm)

## 0. Setup & synthetic data

We'll create a small nonlinear target so the hidden layer is actually useful.

In [5]:
import numpy as np

# Reproducibility
rng = np.random.default_rng(42)

# Create synthetic data: N samples, 2 features
N = 200
x1 = rng.uniform(-2.0, 2.0, size=N)
x2 = rng.uniform(-2.0, 2.0, size=N)
X = np.stack([x1, x2], axis=1)  # shape (N, 2)

# Nonlinear target with a bit of noise
noise = rng.normal(0.0, 0.1, size=N)
y = 1.2 * np.sin(x1) + 0.7 * x2 - 0.3 * x1 * x2 + noise
y = y.reshape(y.size, 1)  # shape (N, 1)

X.shape, y.shape

# Now, let's plot it - this is a rotatable 3-D plot, drag it around to get familiar with the data
import plotly.express as px
import pandas as pd

# Wrap into a DataFrame for nicer plotting
df = pd.DataFrame({"x1": X[:,0], "x2": X[:,1], "y": y.squeeze()})

fig = px.scatter_3d(
    df, x="x1", y="x2", z="y",
    color="y", opacity=0.7,
    title="Interactive 3D view of the dataset"
)
fig.update_traces(marker=dict(size=4))
fig.show()



## 1. Let's Define the loss
### Since we are predicting a single number, our old friend MSE make sense

For a single example with prediction $ \hat{y} $ and true value $ y $:
$$
L = \tfrac{1}{2}(y - \hat{y})^2
$$

For a batch of \(N\) examples:
$$
L = \frac{1}{2N}\sum_{i=1}^{N} \left(y^{(i)} - \hat{y}^{(i)}\right)^2
$$

You probably noticed a difference with the previous MSE:  $ \tfrac{1}{2} $ . This is a nice little trick tat makes the math cleaner - when we take a derivative of something that's squared, we end up with something that's 2x, so if we already have $ \tfrac{1}{2} $, these two conveniently cancel.

In [6]:
def mse_loss(y_pred: np.ndarray, y_true: np.ndarray) -> float:
    """
    Mean Squared Error with a 1/2 factor:
    """
    return 0.5 * np.square(y_pred - y_true).mean()

## lets test it real quick
y_pred_fake = np.array([1, 2, 3, 4, 5])
y_actual_fake = np.array([1, 2, 3, 4, 5])

assert 0.0 == mse_loss(y_pred_fake, y_actual_fake)


y_pred_fake = np.array([1, 2, 3, 4, 5])
y_actual_fake = np.array([2, 4, 6, 8, 10])

# 1 + 4 + 9 + 16 + 25 = 55 / (2 * 5) = 5.5

assert 5.5 == mse_loss(y_pred_fake, y_actual_fake)

print("MSE Test Success.")





MSE Test Success.


## 2. Forward pass (with biases via augmentation
We’ll absorb biases by **augmenting** inputs with a constant 1.

- Original input **X** has 2 features, so shape is `(N, 2)`.
- $N$ in this case could be the number of datapoints we run through the network at the same time
- During training N could be the number of datapoints of the entire datasets
- More commonly N would be the number of data points in the training mini-batch 
- Augmented input **X** = `[X  1]` (append a column of ones), so shape is `(N, 3)`.  
  From here on, **X** means the augmented one.

### First layer (input to hidden, 4 neurons)

- Weights `W[1]` has shape `(3, 4)`:
  3 rows = augmented inputs, 4 columns = 4 neurons in the first hidden layer

- Pre-activations:
$$
Z^{[1]} = X\, W^{[1]}
$$ 
- $Z$ is (N by 4), since $X$ is (N by 3) and we dotted it with $W^{[1]}$ which is (3 by 4)


- Activations:
$$
H = \sigma\!\left(Z^{[1]}\right)
$$

### Hidden layer augmentation

- Now since we are feeding the activations to another weight matrix, the one for the output layer, and we also want to have a bias unit for the output layer, we do the same augmentation trick for the activations output by our hidden layer
$$
H = [\,H\;\;1\,] \quad \in \mathbb{R}^{N \times 5}
$$

### Output layer (hidden to output, linear)

- Weights `W[2]` has shape `(5, 1)`.

- Predictions:
$$
\hat{y} = H\, W^{[2]} \quad \in \mathbb{R}^{N \times 1}
$$

### Sigmoid
$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$

#### Shape cheat-sheet
    X: (N, 3)
    
    W[1]: (3, 4)
    
    Z[1], H: (N, 4)
    
    # Then we augment H
    
    H: (N, 5)
    
    W[2]: (5, 1)
    
    y_hat: (N, 1)
    
#### Neural Net Schematic
![Neural net diagram](04_neural_net.png)


In [7]:
def augment_with_ones(X: np.ndarray) -> np.ndarray:
    '''Append a column of ones to X. If X is (N, d), returns (N, d+1).'''
    ones = np.ones((X.shape[0], 1), dtype=X.dtype)
    return np.hstack([X, ones])

X_test= np.array([
    [1, 2],
    [3, 4],
    [5, 6]
])

# print("Testing augment_with_ones")
# print(f"X_test.shape: {X_test.shape}")
# X_test_augmented = augment_with_ones(X_test)
# print(f"X_test_augmented.shape: {X_test_augmented.shape}")
# print("========")
# print("X_test:")
# print(X_test)
# print("========")
# print("X_test_augmented:")
# print(X_test_augmented)


def sigmoid(z: np.ndarray) -> np.ndarray:
    return 1.0 / (1.0 + np.exp(-z))

# print("========")
# H_sigmoid_test = np.array([-10, -5, -1, 0.1, 0.25, 0.5, 1, 5])
# print("Sigmoid test for: ", H_sigmoid_test)
# np.set_printoptions(suppress=True)
# print(sigmoid(H_sigmoid_test))

def init_params(rng: np.random.Generator = np.random.default_rng(0), scale: float = 0.1):
    '''Initialize weights for 2 to 4 to 1 MLP with bias via augmentation.'''
    W1 = rng.normal(0.0, scale, size=(3, 4))  # (in+1=3) x (hidden=4)
    W2 = rng.normal(0.0, scale, size=(5, 1))  # (hidden+1=5) x (out=1)
    return {"W1": W1, "W2": W2}

# print("========")
# print("Test init_params:")
# print(init_params())

def forward(X: np.ndarray, params: dict):
    '''
    Forward pass.
    X: shape (N, 2)
    Returns: y_hat, cache
    '''
    W1, W2 = params["W1"], params["W2"]
    X_aug = augment_with_ones(X)           # (N, 3)
    Z1 = X_aug @ W1                        # (N, 4)
    H = sigmoid(Z1)                        # (N, 4)
    H_aug = augment_with_ones(H)           # (N, 5)
    y_hat = H_aug @ W2                     # (N, 1) linear output
    cache = {"X_aug": X_aug, "Z1": Z1, "H": H, "H_aug": H_aug, "y_hat": y_hat}
    return y_hat, cache

# Try a dry run
params = init_params(rng)
y_hat, cache = forward(X, params)
print("Shapes -> X:", X.shape, "| X_aug:", cache["X_aug"].shape, "| W1:", params["W1"].shape, 
      "| Z1:", cache["Z1"].shape, "| H:", cache["H"].shape, "| H_aug:", cache["H_aug"].shape, "| W2:", params["W2"].shape, "| y_hat:", y_hat.shape)
print("Initial loss (untrained):", mse_loss(y, y_hat))

Shapes -> X: (200, 2) | X_aug: (200, 3) | W1: (3, 4) | Z1: (200, 4) | H: (200, 4) | H_aug: (200, 5) | W2: (5, 1) | y_hat: (200, 1)
Initial loss (untrained): 0.8242660887048511


## 3. Backpropagation (baby steps)

So far we know how to **predict** with our network:

1. Take the inputs $X$ (augmented with a bias column).
2. Multiply by the first weight matrix $W^{[1]}$.
3. Apply the activation function $\sigma$ to get hidden activations $H$.
4. Augment $H$ with a bias column.
5. Multiply by the second weight matrix $W^{[2]}$.
6. Get the prediction $\hat{y}$.

In short, our forward pipeline looks like this:

$$
X \;\;\xrightarrow{\; W^{[1]} \;}\;\; Z^{[1]} \;\;\xrightarrow{\;\sigma\;}\;\; H
\;\;\xrightarrow{\;\text{augment}\;}\;\; H_{\text{aug}}
\;\;\xrightarrow{\; W^{[2]} \;}\;\; \hat{y}
$$

---

### Why backprop?

We also have a **loss function** (MSE):

$$
L = \frac{1}{2N} \sum_{i=1}^N (y^{(i)} - \hat{y}^{(i)})^2
$$

And your intuition is probably already telling you:  
- If we can compute the **gradient** of this loss with respect to the weights,  
- Then we can **update the weights** in the right direction to reduce the loss next time.

That means:
- First we’ll need to find the derivative of the loss function, $L$ with respect to $W^{[2]}$ (the output layer weights):  $\frac{\partial L}{\partial W^{[2]}}$.  
- Then we’ll also need to find the derivative of the loss function, $L$ with respect to $W^{[1]}$ (the hidden layer weights):  $\frac{\partial L}{\partial W^{[1]}}$, since they influence $\hat{y}$ indirectly.

This is what backpropagation gives us:  
a systematic way to push the error signal backwards through the network, layer by layer, using the chain rule.


## 3a) Backprop: $\frac{\partial L}{\partial W^{[2]}}$

**Recall the forward bits we need:**
- $H_{\text{aug}} \in \mathbb{R}^{N\times 5}$ (4 hidden activations + a column of 1s for bias)
- $W^{[2]} \in \mathbb{R}^{5\times 1}$
- $\hat{y} = H_{\text{aug}}\,W^{[2]} \in \mathbb{R}^{N\times 1}$

**Loss (MSE with $\tfrac12$):**
$$
L = \frac{1}{2N}\sum_{i=1}^N \bigl(\hat{y}^{(i)} - y^{(i)}\bigr)^2
$$

**As in earlier notebooks, we can derive the gradient for a single datapoint and then average. So ignore the sum for a moment and use:**
$$
L = \tfrac{1}{2}\,(\hat{y} - y)^2
$$

### Step 1: gradient w.r.t. prediction (single datapoint)
$$
\frac{\partial L}{\partial \hat{y}} \;=\; \hat{y} - y
$$

### Step 2: chain rule to get gradient w.r.t. $W^{[2]}$ (single datapoint)
Since $\hat{y} = H_{\text{aug}}\,W^{[2]}$ and (for one example) $H_{\text{aug}}$ is a row $(1\times 5)$,
$$
\frac{\partial \hat{y}}{\partial W^{[2]}} \;=\; H_{\text{aug}}^\top \quad (5\times 1)
$$
Therefore,
$$
\frac{\partial L}{\partial W^{[2]}}
\;=\;
\frac{\partial L}{\partial \hat{y}}
\cdot
\frac{\partial \hat{y}}{\partial W^{[2]}}
\;=\;
(\hat{y} - y)\,H_{\text{aug}}^\top
\quad\in\mathbb{R}^{5\times 1}.
$$

### Turn it into the batch average
Over $N$ examples:
$$
\boxed{\;
\frac{\partial L}{\partial W^{[2]}}
\;=\;
\frac{1}{N}\,H_{\text{aug}}^{\!\top}\,(\hat{y}-y)
\;}
\quad\text{with}\quad
H_{\text{aug}}\in\mathbb{R}^{N\times 5},\;
(\hat{y}-y)\in\mathbb{R}^{N\times 1}.
$$

**Shape check:** $(5\times N)\cdot(N\times 1)=(5\times 1)$


In [16]:
# y_hat, H_aug from the forward pass (e.g., via cache)
y_hat = cache["y_hat"]     # (N, 1)
H_aug = cache["H_aug"]     # (N, 5)

N = y.shape[0]
dL_dy_hat = (y_hat - y) / N      # (N, 1)
grad_W2   = H_aug.T @ dL_dy_hat  # (5, 1)

print(dL_dy_hat.shape)  # (N, 1)
print(grad_W2.shape)    # (5, 1)

(200, 1)
(5, 1)


#### Well, kewl, now we have the loss gradient with respect with W_2
That's enough information to allows to update the weights in $W^{[2]}$:

In [20]:
learning_rate = 0.05
w2_before_update = params["W2"]
assert np.array_equal(params["W2"], w2_before_update)
params["W2"] = params["W2"] - learning_rate * grad_W2
assert not np.array_equal(params["W2"], w2_before_update)

### Baby Step 3 — Translate hidden error into updates for $W^{[1]}$

Now that each hidden neuron has its **error signal** (from Baby Step 2),  
we can finally connect that signal back to the weights $W^{[1]}$ that created those hidden activations.

Here’s the intuition:

- Each hidden neuron $h_j$ is the result of a **weighted sum of inputs** (plus bias):  
  $$
  z^{[1]}_j = x_1 \cdot W^{[1]}_{1,j} \;+\; x_2 \cdot W^{[1]}_{2,j} \;+\; 1 \cdot W^{[1]}_{\text{bias},j}
  $$

- If a hidden neuron ended up contributing to the output error,  
  then all of the weights that fed into it should be nudged a little.

- How much should each weight change? Two factors matter:
  1. **The input value** that came through that connection (if an input was 0, that weight didn’t matter this time).  
  2. **The hidden error** for that neuron (if the neuron had no error signal, its weights don’t need to move).

Put together:

> **The gradient for $W^{[1]}$ is basically the input values multiplied by the hidden errors.**

In matrix form, this becomes an outer product:

$$
\frac{\partial L}{\partial W^{[1]}} = X_{\text{aug}}^\top \cdot \text{hidden\_error}
$$

- $X_{\text{aug}}$: input with the bias column, shape $(N \times 3)$  
- $\text{hidden\_error}$: error signals for the 4 hidden neurons, shape $(N \times 4)$  
- Result: gradient for $W^{[1]}$, shape $(3 \times 4)$

For completeness, we can substitute the hidden\_error term from Baby Step 2:

$$
\frac{\partial L}{\partial W^{[1]}}
= X_{\text{aug}}^\top \cdot \Bigl( \bigl((\hat{y} - y) \times W^{[2]}_{\text{(no bias)}}\bigr) \;\times\; (H \cdot (1 - H)) \Bigr)
$$

This gradient tells us exactly how to nudge each input-to-hidden weight so that next time,  
the hidden activations $H$ will be a little closer to what we wished they were.

---

### Minimal NumPy snippet


In [25]:
# From the forward pass / cache:
X_aug = cache["X_aug"]      # (N, 3)
H      = cache["H"]         # (N, 4)
y_hat  = cache["y_hat"]     # (N, 1)
W2     = params["W2"]       # (5, 1)

N = y.shape[0]

# Hidden blame (drop the bias weight from W2)
hidden_blame = (y_hat - y) @ W2[:-1].T      # (N, 4)

# Sigmoid sensitivity
sigmoid_sensitivity = H * (1 - H)           # (N, 4)

# Hidden error
hidden_error = hidden_blame * sigmoid_sensitivity   # (N, 4)

# Gradient for W1
grad_W1 = X_aug.T @ hidden_error            # (3, 4)

print(grad_W1.shape)  # (3, 4)


## And now we could "update" W1 if we wanted to
w1_before_update = params["W1"]
assert np.array_equal(params["W1"], w1_before_update)
params["W1"] = params["W1"] - learning_rate * grad_W1
assert not np.array_equal(params["W1"], w1_before_update)

(3, 4)



# Formal backprop derivation (optional, specific to our network)

---

# Step 1 — Output layer (matrix style, but still using fractions)

**Prediction:**

$$
\hat y = W_2 \, h
$$

where

* $W_2$ is a $1 \times 5$ row vector of weights
* $h$ is a $5 \times 1$ column vector of hidden activations (with the bias “1” included)

So $\hat y$ is just a scalar.

---

**Loss:**

$$
L = \tfrac{1}{2}(\hat y - y)^2
$$

---

### Step 1A: derivative of the loss w\.r.t. the prediction

$$
\frac{\partial L}{\partial \hat y} = \hat y - y
$$

---

### Step 1B: derivative of the prediction w\.r.t. the weight vector

Since

$$
\hat y = W_2 h,
$$

we can view $\hat y$ as a dot product of the row vector $W_2$ with the column vector $h$.

The derivative of a dot product w\.r.t. the row vector is just the column vector transposed:

$$
\frac{\partial \hat y}{\partial W_2} = h^\top
$$

---

### Why is it $h^\top$ and not just $h$?

* $W_2$ is a row vector of shape $1 \times 5$.
* The gradient must have the same shape.
* Writing $h^\top$ (a row vector) ensures the shapes line up: both are $1 \times 5$.
* If we wrote just $h$ (a column), the shapes wouldn’t match.

---

### Step 1C: chain rule

Now combine everything:

$$
\frac{\partial L}{\partial W_2}
= \frac{\partial L}{\partial \hat y} \cdot \frac{\partial \hat y}{\partial W_2}
= (\hat y - y) \, h^\top
$$




---

# Step 2 — Hidden layer

**Hidden activations:**

$$
h = \sigma(z_1), \quad z_1 = W_1 x
$$

where

* $W_1$ is a $4 \times 3$ weight matrix (2 inputs + bias, 4 hidden units)
* $x$ is a $3 \times 1$ input vector (augmented with bias “1”)
* $h$ is a $4 \times 1$ column of sigmoid outputs, later augmented with a bias “1” before going to the output layer

---

### Step 2A: derivative of the loss w\.r.t. the hidden activations

From Step 1 we had

$$
\frac{\partial L}{\partial \hat y} = \hat y - y
$$

And $\hat y = W_2 h$.

So,

$$
\frac{\partial \hat y}{\partial h} = W_2^\top
$$

Thus,

$$
\frac{\partial L}{\partial h} = \frac{\partial L}{\partial \hat y} \cdot \frac{\partial \hat y}{\partial h}
= (\hat y - y) \, W_2^\top
$$

---

### Step 2B: derivative of hidden activations w\.r.t. pre-activations

Each hidden unit uses the sigmoid:

$$
h = \sigma(z_1), \quad \sigma'(z) = \sigma(z)(1 - \sigma(z))
$$

So,

$$
\frac{\partial h}{\partial z_1} = h \odot (1 - h)
$$

(where $\odot$ means elementwise product).

---

### Step 2C: chain rule

$$
\frac{\partial L}{\partial z_1} =
\left[ \frac{\partial L}{\partial h} \right] \odot \left[ \frac{\partial h}{\partial z_1} \right]
= \big( (\hat y - y) \, W_2^\top \big) \odot \big(h \odot (1-h)\big)
$$

---

This gives us the “hidden error signal” (often written $\delta_1$):

$$
\delta_1 = \big( (\hat y - y) \, W_2^\top \big) \odot \big(h \odot (1-h)\big)
$$

---

Next in **Step 3**, we’ll use this $\delta_1$ to compute the gradient w\.r.t. the first-layer weights $W_1$.



Awesome—straight to it.

# Step 3 - First layer weights $W_1$

**Setup:**

$$
z_1 = W_1\,x,\qquad h=\sigma(z_1),\qquad \delta_1=\Big((\hat y-y)\,W_2^\top\Big)\odot\big(h\odot(1-h)\big)
$$

* $W_1 \in \mathbb{R}^{4\times 3}$ (2 inputs + bias), $x\in\mathbb{R}^{3\times 1}$.
* $\delta_1 \in \mathbb{R}^{4\times 1}$ corresponds to the 4 hidden neurons (not the bias unit).

---

### 3A) Derivative of $z_1$ w\.r.t. $W_1$

Each hidden pre-activation is linear in $W_1$:

$$
z_1 = W_1 x \;\;\Longrightarrow\;\; \frac{\partial z_1}{\partial W_1} \text{ acts like “multiply by } x^\top\text{”.}
$$

Concretely, for any entry $W_{1,ik}$, $\frac{\partial z_{1,i}}{\partial W_{1,ik}} = x_k$.

---

### 3B) Chain rule to get $\frac{\partial L}{\partial W_1}$

$$
\frac{\partial L}{\partial W_1}
= \frac{\partial L}{\partial z_1}\;\frac{\partial z_1}{\partial W_1}
= \delta_1 \; x^\top
$$

That’s an **outer product**: column $\delta_1$ times row $x^\top$.

---

### 3C) Shapes (quick check)

* $\delta_1$ is $4\times 1$
* $x^\top$ is $1\times 3$
* $\delta_1 x^\top$ is $4\times 3$  - matches $W_1$

---

### Mini-batch version (mean over $N$ examples)

$$
\frac{\partial \bar L}{\partial W_1} \;=\; \frac{1}{N}\sum_{n=1}^N \delta_1^{(n)} \,{x^{(n)}}^\top
$$

---

### Tiny intuition

* Each weight $W_{1,ik}$ connects input feature $x_k$ into hidden neuron $i$.
* Its update is proportional to **(error at that hidden neuron)** $\delta_{1,i}$ times **(that input feature)** $x_k$.

---
